In [ ]:
## in this session i apply the resnet50 ,
## i train the utkface data (its train set) and then i give the accuracy and loss for training and validation (near 70%)
## and then i test it to the spesific_dataset (55%)
## at the end i predict some pictures

In [ ]:
from google.colab import files
upload = files.upload()

Saving utkface_train.zip to utkface_train.zip


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving specific_dataset.zip to specific_dataset.zip


In [ ]:
import zipfile
import os

# Specify the name of the uploaded zip file
zip_file_name = "utkface_train.zip"

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("utkface_dataset")

In [ ]:
import zipfile
import os

# Specify the name of the uploaded zip file
zip_file_name = "specific_dataset.zip"

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("specific_dataset")

In [ ]:
## removing cropted images

In [ ]:
import os
import cv2

# Define the directory containing your dataset
dataset_dir = "/content/specific_dataset/specific_dataset"

# Function to check if an image file is corrupted
def is_corrupted_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return True
        else:
            return False
    except:
        return True

# List to store the names of corrupted images
corrupted_images = []

# Iterate through the dataset directory and subdirectories
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            # Construct the full path to the image file
            image_path = os.path.join(root, file)
            # Check if the image is corrupted
            if is_corrupted_image(image_path):
                corrupted_images.append(image_path)
                # Remove the corrupted image file
                os.remove(image_path)

# Print the names of corrupted images
print("Corrupted images:")
for image_path in corrupted_images:
    print(image_path)

Corrupted images:


In [ ]:
# Define the directory containing your dataset
dataset_dir = "/content/utkface_dataset/utkface_train"

# Function to check if an image file is corrupted
def is_corrupted_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return True
        else:
            return False
    except:
        return True

# List to store the names of corrupted images
corrupted_images = []

# Iterate through the dataset directory and subdirectories
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            # Construct the full path to the image file
            image_path = os.path.join(root, file)
            # Check if the image is corrupted
            if is_corrupted_image(image_path):
                corrupted_images.append(image_path)
                # Remove the corrupted image file
                os.remove(image_path)

# Print the names of corrupted images
print("Corrupted images:")
for image_path in corrupted_images:
    print(image_path)

Corrupted images:


In [ ]:
## preparing the datasets

In [ ]:
import torch

In [ ]:
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

# Define transformations for both datasets
train_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),  # Resize images to fit pre-trained models input size
    transforms.RandomHorizontalFlip(),   # Randomly flip images horizontally for data augmentation
    transforms.ToTensor(),               # Convert images to tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize images
])

test_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load datasets
utkface_dataset = ImageFolder('/content/utkface_dataset/utkface_train', transform=train_transform)
specific_dataset = ImageFolder('/content/specific_dataset/specific_dataset', transform=test_transform)

# Split UTKFace dataset into training and validation sets (you can use train_transform for both)
train_size = int(0.8 * len(utkface_dataset))
val_size = len(utkface_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(utkface_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(specific_dataset, batch_size=32)


In [ ]:
# Choose pre-trained CNN models: ResNet50

In [ ]:
import torchvision.models as models

# Initialize pre-trained ResNet50 model
pretrained_model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s]


In [ ]:
## Fine-tune the pre-trained models:

In [ ]:
import torch.nn as nn

ethnicity_classes = ['asian', 'black' , 'indian', 'white']

# Freeze all pre-trained layers
for param in pretrained_model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer
num_classes = len(ethnicity_classes)  # Number of ethnicity classes
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, num_classes)

In [ ]:
## train the model

In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    pretrained_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = pretrained_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model = pretrained_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = pretrained_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate training loss and accuracy
    epoch_loss = running_loss / len(train_dataset)
    train_accuracy = correct_train / total_train

    # Validation phase
    pretrained_model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = pretrained_model(images)
            val_loss = criterion(outputs, labels)
            running_val_loss += val_loss.item() * images.size(0)

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate validation loss and accuracy
    val_loss = running_val_loss / len(val_dataset)
    val_accuracy = correct_val / total_val

    # Print epoch results
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {epoch_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, '
          f'Val Loss: {val_loss:.4f}, '
          f'Val Accuracy: {val_accuracy:.4f}')

Epoch [1/10], Train Loss: 0.8424, Train Accuracy: 0.6628, Val Loss: 0.8080, Val Accuracy: 0.6730
Epoch [2/10], Train Loss: 0.8176, Train Accuracy: 0.6749, Val Loss: 0.8124, Val Accuracy: 0.6730
Epoch [3/10], Train Loss: 0.8021, Train Accuracy: 0.6839, Val Loss: 0.8427, Val Accuracy: 0.6669
Epoch [4/10], Train Loss: 0.7930, Train Accuracy: 0.6904, Val Loss: 0.7836, Val Accuracy: 0.6923
Epoch [5/10], Train Loss: 0.7786, Train Accuracy: 0.6907, Val Loss: 0.8209, Val Accuracy: 0.6752
Epoch [6/10], Train Loss: 0.7623, Train Accuracy: 0.7008, Val Loss: 0.8437, Val Accuracy: 0.6737
Epoch [7/10], Train Loss: 0.7629, Train Accuracy: 0.7053, Val Loss: 0.7718, Val Accuracy: 0.6961
Epoch [8/10], Train Loss: 0.7608, Train Accuracy: 0.7059, Val Loss: 0.7794, Val Accuracy: 0.6877
Epoch [9/10], Train Loss: 0.7463, Train Accuracy: 0.7094, Val Loss: 0.7710, Val Accuracy: 0.6953
Epoch [10/10], Train Loss: 0.7415, Train Accuracy: 0.7132, Val Loss: 0.7693, Val Accuracy: 0.6919


In [ ]:
## plot the training and validation results (e.g., loss and accuracy)

In [ ]:
print(epochs)

range(1, 11)


In [ ]:
import matplotlib.pyplot as plt

# Plotting
#epochs = range(1, num_epochs + 1)
epochs = range(0, 1)


plt.figure(figsize=(12, 5))

# Plotting losses
plt.subplot(1, 2, 1)
plt.plot(epochs, epoch_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting accuracies
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
print(epoch_loss, val_loss, train_accuracy, val_accuracy)


0.7414627805683661 0.769300718590854 0.7132039753904401 0.691900075700227


In [ ]:
# Rescale the values from the range [0, 1] to the range [1, 10]
def scale_value(value):
    return (value * 9) + 1

# Convert the float values to a single-element list and then apply scaling
train_loss_values_scaled = [scale_value(epoch_loss)]
val_loss_values_scaled = [scale_value(val_loss)]
train_accuracy_values_scaled = [scale_value(train_accuracy)]
val_accuracy_values_scaled = [scale_value(val_accuracy)]

In [ ]:
def plot_training_results(epoch_loss_values, val_loss_values, train_accuracy_values, val_accuracy_values):
    epochs = range(1, len(epoch_loss_values) + 1)

    # Plot losses
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss_values_scaled, label='Training Loss')
    plt.plot(epochs, val_loss_values_scaled, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracies
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy_values_scaled, label='Training Accuracy')
    plt.plot(epochs, val_accuracy_values_scaled, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage
plot_training_results(train_loss_values_scaled, val_loss_values_scaled, train_accuracy_values_scaled, val_accuracy_values_scaled)

In [ ]:
# Plotting
epochs = range(1, num_epochs + 1)
plt.figure(figsize=(12, 4))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss_values_scaled, label='Training Loss')
plt.plot(epochs, val_loss_values_scaled, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy_values_scaled, label='Training Accuracy')
plt.plot(epochs, val_accuracy_values_scaled, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

#### after running this code i try to rescale them to show the plot above this code but the values don't show properly.

In [ ]:
## evaluate the model on a test set

In [ ]:
correct = 0
total = 0
pretrained_model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = pretrained_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on test dataset:', accuracy)

Accuracy on test dataset: 0.5518672199170125


In [ ]:
##  Predict ethnicity on new images

In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/asian/Claudia Kim.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)


## image is asian

Predicted ethnicity: asian
Predicted percentages: {'asian': 88.81845474243164, 'black': 3.3992674201726913, 'indian': 2.011064626276493, 'white': 5.77121265232563}


In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/white/Kit Harington.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is white

Predicted ethnicity: white
Predicted percentages: {'asian': 1.2015914544463158, 'black': 6.694620847702026, 'indian': 3.5400226712226868, 'white': 88.56377005577087}


In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/black/Screenshot 2024-04-17 at 13.17.56.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is black

Predicted ethnicity: black
Predicted percentages: {'asian': 0.854259543120861, 'black': 74.00350570678711, 'indian': 0.06894152029417455, 'white': 25.07328987121582}


In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/indian/Abhishek Bachchan 2.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is indian

Predicted ethnicity: indian
Predicted percentages: {'asian': 10.207272320985794, 'black': 34.42340791225433, 'indian': 39.27358388900757, 'white': 16.09574407339096}


In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/asian/Screenshot 2024-04-17 at 14.11.01.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is asian

Predicted ethnicity: white
Predicted percentages: {'asian': 18.81915181875229, 'black': 0.057055585784837604, 'indian': 1.3388642109930515, 'white': 79.78492975234985}


In [ ]:
from PIL import Image

from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/asian/Screenshot 2024-04-17 at 14.11.01.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is asian

Predicted ethnicity: white
Predicted percentages: {'asian': 18.81915181875229, 'black': 0.057055585784837604, 'indian': 1.3388642109930515, 'white': 79.78492975234985}


In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]
    return predicted_ethnicity

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/asian/Henry Golding.png'
predicted_ethnicity = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)

## image is asian

Predicted ethnicity: white


In [ ]:
from PIL import Image

def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/asian/Henry Golding.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is asian

Predicted ethnicity: white
Predicted percentages: {'asian': 31.166130304336548, 'black': 14.783936738967896, 'indian': 5.709195509552956, 'white': 48.34073185920715}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/black/257.jpg'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is black

Predicted ethnicity: black
Predicted percentages: {'asian': 1.124653685837984, 'black': 98.5909640789032, 'indian': 0.10365231428295374, 'white': 0.180733110755682}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/black/148.jpg'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is black

Predicted ethnicity: black
Predicted percentages: {'asian': 0.09833123767748475, 'black': 99.00512099266052, 'indian': 0.006930356175871566, 'white': 0.8896089158952236}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/indian/Aamir Khan 4.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is indian

Predicted ethnicity: indian
Predicted percentages: {'asian': 3.1220410019159317, 'black': 0.7477778475731611, 'indian': 48.65243434906006, 'white': 47.477737069129944}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/indian/Chitrangada Singh.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is indian

Predicted ethnicity: white
Predicted percentages: {'asian': 1.4847097918391228, 'black': 19.7967991232872, 'indian': 14.087319374084473, 'white': 64.63117599487305}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = "/content/specific_dataset/specific_dataset/indian/Ileana D'Cruz.png"
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is indian

Predicted ethnicity: white
Predicted percentages: {'asian': 1.615379936993122, 'black': 1.5957033261656761, 'indian': 1.8101796507835388, 'white': 94.9787437915802}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/white/Brad Pitt.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is white

Predicted ethnicity: white
Predicted percentages: {'asian': 0.8296683430671692, 'black': 0.2810451900586486, 'indian': 0.11875720228999853, 'white': 98.77053499221802}


In [ ]:
def predict_ethnicity(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0)

    # Predict ethnicity
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_class = predicted.item()
    predicted_ethnicity = ethnicity_classes[predicted_class]

    # Calculate percentage of each predicted label
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_percentages = {ethnicity_classes[i]: probabilities[i].item() * 100 for i in range(len(ethnicity_classes))}

    return predicted_ethnicity, predicted_percentages

# Example usage:
image_path = '/content/specific_dataset/specific_dataset/white/Screenshot 2024-04-17 at 14.43.46.png'
predicted_ethnicity, predicted_percentages = predict_ethnicity(image_path, pretrained_model)
print('Predicted ethnicity:', predicted_ethnicity)
print('Predicted percentages:', predicted_percentages)

## image is white

Predicted ethnicity: white
Predicted percentages: {'asian': 0.46131573617458344, 'black': 18.014568090438843, 'indian': 1.7358886078000069, 'white': 79.78822588920593}
